In [1]:
import os
from dotenv import load_dotenv
import openai

_ = load_dotenv(override=True)

# LLM Chain

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [5]:
llm = ChatOpenAI(
    api_key = os.getenv("OPEN_AI_KEY"),
    model = os.getenv("LLM_MODEL")
)

prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe a company that makes {product}"
)

chain = LLMChain(llm=llm, prompt=prompt)

product = "podcast about data science in typical real life scenarios"
chain.run(product)

'DataStories'

# Sequential Chain
Comine multiple Chains where the **output** of one chain is the **input** of another

## 1) Simple Sequential Chain
Single Input Single output

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import SimpleSequentialChain

In [8]:
llm = ChatOpenAI(
    api_key = os.getenv("OPEN_AI_KEY"),
    model = os.getenv("LLM_MODEL")
)

# First Chain
prompt_1 = ChatPromptTemplate.from_template(
    "What is the best name to describe a company that makes {product}"
)
chain_1 = LLMChain(llm=llm, prompt=prompt_1)

# Second Chain
prompt_2 = ChatPromptTemplate.from_template(
    "write a 20 words description for the following company: {company}"
)
chain_2 = LLMChain(llm=llm, prompt=prompt_2)

# Join chains
full_chain = SimpleSequentialChain(chains=[chain_1, chain_2], verbose=True)
full_chain.run("podcast about data science in typical real life scenarios")



> Entering new SimpleSequentialChain chain...
"DataSpeak" or "DataSage"
DataSpeak or DataSage is a cutting-edge tech company specializing in advanced data analytics and providing actionable insights for businesses.

> Finished chain.


'DataSpeak or DataSage is a cutting-edge tech company specializing in advanced data analytics and providing actionable insights for businesses.'

## 2) Sequential Chain
multiple Input multiple output

In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import SequentialChain

In [13]:
llm = ChatOpenAI(
    api_key = os.getenv("OPEN_AI_KEY"),
    model = os.getenv("LLM_MODEL")
)

# First Chain
prompt_1 = ChatPromptTemplate.from_template(
    "What is the best name to describe a company that makes {product}"
)
chain_1 = LLMChain(llm=llm, prompt=prompt_1, output_key="name")

# Second Chain
prompt_2 = ChatPromptTemplate.from_template(
    "write a 20 words description for the following company: {name}"
)
chain_2 = LLMChain(llm=llm, prompt=prompt_2, output_key="description")

# Third Chain
prompt_3 = ChatPromptTemplate.from_template(
    "Create a Slogan for a company named '{name}' that is specialized in: {product}"
)
chain_3 = LLMChain(llm=llm, prompt=prompt_3, output_key="slogan")

# Fourth Chain
prompt_4 = ChatPromptTemplate.from_template(
    "Make a list of 3 items offered by a company that do {product}"
)
chain_4 = LLMChain(llm=llm, prompt=prompt_4, output_key="items")

# fifth Chain
prompt_5 = ChatPromptTemplate.from_template(
    "Make a short text for an advertizement for the company '{name}' ({slogan}) that offers {items} "
)
chain_5 = LLMChain(llm=llm, prompt=prompt_5, output_key="ad")



# Join chains
full_chain = SequentialChain(
    chains=[chain_1, chain_2,chain_3,chain_4,chain_5],
    input_variables=["product"],
    output_variables=["name", "description","slogan","items","ad"],
    verbose=True
)
full_chain("podcast about data science in typical real life scenarios")




> Entering new SequentialChain chain...

> Finished chain.


{'product': 'podcast about data science in typical real life scenarios',
 'name': '"DataSage"',
 'description': 'DataSage: A cutting-edge data analytics company providing businesses with advanced insights and solutions to drive growth and maximize efficiency.',
 'slogan': '"Unlocking the Power of Data Science, One Story at a Time!"',
 'items': "1. Expert Interviews: The company offers podcast episodes featuring interviews with leading data science professionals and industry experts. These interviews cover a wide range of real-life scenarios and discuss how data science is applied in various industries such as healthcare, finance, marketing, and more. Listeners can gain insights into the challenges faced, practical solutions employed, and the impact of data science in solving complex problems.\n\n2. Case Studies: The podcast also presents detailed case studies that dive into real-life data science projects. These case studies explore how data science techniques were employed to analyze 

## 3) Router Chain
Address the result of a prompt to a specific chain

In [40]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router import MultiPromptChain
from langchain.chains import LLMChain

In [32]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [33]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [34]:
llm = ChatOpenAI(
    api_key = os.getenv("OPEN_AI_KEY"),
    model = os.getenv("LLM_MODEL")
)

In [44]:
destination_chains = {}
destinations = []

for e in prompt_infos:
    name = e["name"]
    template = e["prompt_template"]
    description = e["description"]
    prompt = ChatPromptTemplate.from_template(template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain
    destinations.append(f"{name}: {description}")

default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

destinations_str = "\n".join(destinations)
    

In [51]:
MULTI_PROMPT_ROUTER_TEMPLATE = """
Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}```


REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [52]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser()
)

router_chain = LLMRouterChain.from_llm(llm=llm, prompt=router_prompt)

In [54]:
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain
)

In [55]:
chain.run("What is the difference between general and Specific Relativity?")

c:\Users\crisi\Documents\AI Courses\langchain\.venv\lib\site-packages\langchain\chains\llm.py:321: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


'General relativity and special relativity are both theories proposed by Albert Einstein to describe the nature of gravity and the behavior of objects in different frames of reference. \n\nSpecial relativity, also known as the theory of special relativity, applies to situations where there is no acceleration or gravitational force involved. It deals with the behavior of objects moving at constant speeds relative to each other. Special relativity introduces the concept of spacetime, where space and time are intertwined, and the laws of physics are the same for all observers in uniform motion. It also introduces the famous equation E=mc², which relates energy (E) and mass (m).\n\nOn the other hand, general relativity is an extension of special relativity that includes gravity. It describes the force of gravity as the curvature of spacetime caused by mass and energy. In general relativity, the presence of mass and energy warps the fabric of spacetime, and objects move along curved paths i